## Webscraping

In this exercise, you'll practice using BeautifulSoup to parse the content of a web page. The page that you'll be scraping, https://realpython.github.io/fake-jobs/, contains job listings. Your job is to extract the data on each job and convert into a pandas DataFrame.


In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import io


### 1. Start by performing a GET request on the url above and convert the response into a BeautifulSoup object. 

In [17]:
endpoint = 'https://realpython.github.io/fake-jobs/'
response = requests.get(endpoint)
response.status_code
soup = BeautifulSoup(response.text, features="html.parser")

 #### a. Use the .find method to find the tag containing the first job title ("Senior Python Developer"). Hint: can you find a tag type and/or a class that could be helpful for extracting this information? Extract the text from this title. 

In [18]:
s_python_dev = soup.find('h2').text#:'Senior Python Developer')

 #### b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list. 

In [ ]:
job_titles = [item.text for item in soup.findAll('h2')]

 #### c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end. 

In [ ]:
companies = [name.text for name in soup.findAll(attrs={'class':'subtitle is-6 company'})]
locations = [place.text for place in soup.findAll(attrs={'class':'location'})]
locations = [item.strip() for item in locations if str(item)]
dates = [day.text for day in soup.findAll('time')]


['2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-08',
 '2021-04-

#### d. Take the lists that you have created and combine them into a pandas DataFrame. 


In [77]:
job_app_data = pd.DataFrame({'company':companies,'location': locations, 'date':dates})
job_app_data

,company,location,date
0,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08
1,Vasquez-Davidson,"Christopherville, AA",2021-04-08
2,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08
3,Savage-Bradley,"East Seanview, AP",2021-04-08
4,Ramirez Inc,"North Jamieview, AP",2021-04-08
...,...,...,...
95,"Nguyen, Yoder and Petty","Lake Abigail, AE",2021-04-08
96,Holder LLC,"Jacobshire, AP",2021-04-08
97,Yates-Ferguson,"Port Susan, AE",2021-04-08
98,Ortega-Lawrence,"North Tiffany, AA",2021-04-08


### 2. Next, add a column that contains the url for the "Apply" button. Try this in two ways.  

#### a. First, use the BeautifulSoup find_all method to extract the urls. 

#### b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.
    


### 3. Finally, we want to get the job description text for each job.  


#### a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.  


#### b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.". 

#### c. Use the [.apply method](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) on the url column you created above to retrieve the description text for all of the jobs.